### Transformer
- Encoder Self-Attention : 인코더는 입력 문장의 모든 단어 간의 연관성을 계산(Self-Attention), 문장 전체에서 문맥의 정보를 파악
- Add & Norm : 해당 출력에 원래 입력을 더함(Skip Connection) & 값을 정규화
- Feed Forward : 활성화 함수(ReLU) 사용하여 입력을 처리, 비선형성을 구현해서 더 복잡한 패턴을 학습할 수 있도록 만드는 완전연결층
- Masked Decoder Self-Attention : 디코더는 문장을 생성(예측) 하는게 목적, 예측할 때 미래ㅔ에 나오지 않은 단어 들의 Attention score가 반영되지 못하게 마스킹하여, 이전 단어들만 고려해서 순차적으로 예측
- Encoder Decoder Self-Attention : 인코더의 출력을 참조해서, 더 정확한 출력을 생성

### BERT
- Transformer의 인코더 구조 사용 -> 입력된 문장을 정확하게 이해하는 역할을 수행
  - 양방향의 이해
  - 문장 의미 추출이 강점 -> 감정 분석, 마스크 채우기 등의 Task가 효율적
- 주의사항 : BERT는 양방향 특성으로 인해 문장을 이해하는데 탁월한데, 생성 작업에는 적합하지 않음

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/인사교/DL/언어지능

/content/drive/MyDrive/인사교/DL/언어지능


In [3]:
# 허깅페이스 API, torch 옵션 -> PyTouch 백엔드를 사용하는 모델들을 위한 추가 패키지 설치
!pip install transformers
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00


In [4]:
# 데이터 불러오기
from datasets import load_dataset
data = load_dataset('smilegate-ai/kor_unsmile')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


dataset_infos.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

valid-00000-of-00001.parquet:   0%|          | 0.00/290k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15005 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/3737 [00:00<?, ? examples/s]

In [5]:
data

DatasetDict({
    train: Dataset({
        features: ['문장', '여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean', '개인지칭', 'labels'],
        num_rows: 15005
    })
    valid: Dataset({
        features: ['문장', '여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean', '개인지칭', 'labels'],
        num_rows: 3737
    })
})

In [6]:
data['train'][0]

{'문장': '일안하는 시간은 쉬고싶어서 그런게 아닐까',
 '여성/가족': 0,
 '남성': 0,
 '성소수자': 0,
 '인종/국적': 0,
 '연령': 0,
 '지역': 0,
 '종교': 0,
 '기타 혐오': 0,
 '악플/욕설': 0,
 'clean': 1,
 '개인지칭': 0,
 'labels': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]}

- Up stream : 모델이 일반적인 언어 패턴을 배우는 사전 학습 과정
- Down Stream : 사전 학습 된 모델을 특정 작업에 맞게 조정하는 파인튜닝 과정

In [7]:
# 학습에 필요한 클래스들 불러오기
from transformers import BertForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
# BertForSequenceClassification : 문장 분류 작업에 사용되는 BERT 모델기반 클래스
# AutoTokenizer : 사전 학습된 모델에 알맞는 토크나이저를 자동을 ㅗ불러오는 도구
# Trainer : 모델 학습을 담당하는 클래스
# TrainingArguments : 학습에 필요한 하이퍼파라미터(학습률, 배치크기, 에포크 수 등)를 다루는 클래스
# DataCollatorWithPadding : 데이터를 제공하는 클래스(배치 단위, 자동을 패딩)

import numpy as np
import pandas as pd

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

https://huggingface.co/beomi/kcbert-base

In [8]:
# 모델 name 정의 : KcBERT(Korean comments BERT)
model_name = 'beomi/kcbert-base'

# 네이버 뉴스(언론소비율 1등)에서 댓글, 대댓글 수집해서 토크나이저 + BERT 모델을 진행한 사전 학습모델
# 오타, 구어체, 신조어 등에 강함

In [9]:
# 1. 토크나이저 로드
# 토큰화, 수치화 작업 담당 -> BERT 모델의 입력값에 맞게끔 토큰화 및 수치화를 담당

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    clean_up_tokenization_spaces = True, # 기본값  True, 토큰화를 수행하고나서 공백 등을 정리하는 옵션
)

# transformers 4.45버전 이상부터는 기본값이 False로 변경될 예정

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

In [10]:
# 한개 문장 토큰화 테스트
tokenizer(data['train'][0]['문장'])

# token_type_ids : 문장 간의 구분하는 역할
# BERT 모델은 예를들어, 두 문장이 주어질 떄 각 문장을 구분하기 위해 사용되는 값
# 여러 문장이 있는 경우, 첫번째 문장은 0, 두번째 문장은 1번으로 구분 됨

# attention_mask : 입력 문장을 처리할 때, 어떤 토큰에 주목해야 되는지 숫자값으로 표시
# 실제 단어가 있는지 여부(1이면 Attention Score에 사용되어야 함, 0이면 패딩되어 있는 값이거나 모델이 무시해야 될 값)

{'input_ids': [2, 2458, 15751, 24930, 24351, 29278, 17038, 11631, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
data['train'].features.keys()

dict_keys(['문장', '여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean', '개인지칭', 'labels'])

In [12]:
# 학습에 필요한 토크나이징 & y데이터(3개의 클래스) 컬럼 추가하기 위한 함수 정의

def preprocessing(doc) :

  # 토크나이징 -> BERT 모델의 입력을 받을 수 있는 형식
  token_str = tokenizer(
      str(doc["문장"]),
      truncation = True, # 최대 길이를 초과하는 부분은 잘라냄
      max_length = 512 # 최대 시퀀스 길이
  )

  # labels를 3개의 클래스로 변경
  label_list = [
      # 8개에 대한 혐오 표현 카테고리들 중에서 하나라도 1인 경우 1.0으로 설정
      1.0 if any([doc['여성/가족'], doc['남성'], doc['성소수자'], doc['인종/국적'], doc['연령'], doc['지역'], doc['종교'], doc['기타 혐오']]) else 0.0 , # 혐오표현
      float(doc["악플/욕설"]), # 악플/욕설
      float(doc["clean"])# clean
  ]

  # 토큰화 된 문장에 labels라는 필드를 추가
  token_str["labels"] = label_list

  return token_str

In [13]:
# 함수 동작 확인
preprocessing(data['train'][0])

{'input_ids': [2, 2458, 15751, 24930, 24351, 29278, 17038, 11631, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [0.0, 0.0, 1.0]}

In [14]:
# 기존 라벨 확인
data['train'][0]['labels']

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [15]:
# apply() 비슷
tokenized_datasets = data.map(
    preprocessing, # 앞에서 정의한 함수 적용
    remove_columns = data['train'].column_names, # 기존에 컬럼들 제거(충돌 방지용)
)
# map()과 with_transform()
# map() : 변환을 즉시 적용 -> 토큰화, 라벨 변환 등 데이터 전처리 작업에 주로 사용 // 즉시 변환이다보니, 모든 데이터셋을 무조건적으로 변환해야 함
# with_transform() : 변환을 등록해둠 -> 실제 데이터 접근 시 변환을 적용(학습을 수행하면서 호출할 때 변형이 이루어짐) // 메모리 절약이 효율적

Map:   0%|          | 0/15005 [00:00<?, ? examples/s]

Map:   0%|          | 0/3737 [00:00<?, ? examples/s]

In [16]:
type(tokenized_datasets['train'][0]['labels'][0])
# 함수를 적용하고나서 값을 확인해봤더니, 'labels'는 int형!

# 예를 들어, 평가 지표 함수에는 float형태를 기대하는 경우, int형이 안 들어가는 상황도 발생

int

In [17]:
# 데이터셋의 Feature 타입들을 직접 정의
from datasets import Features, Sequence, Value

In [18]:
# 1. 피처타입 세팅
features = Features({
    'input_ids': Sequence(feature = Value(dtype = 'int64')), # 정수형으로(int64)으로 명시해서 지정
    'token_type_ids' : Sequence(feature = Value(dtype = 'int64')),
    'attention_mask' : Sequence(feature = Value(dtype = 'int64')),
    'labels' : Sequence(feature = Value(dtype = 'float32')) # 실수형(float32)으로 명시해서 지정
})

In [19]:
# 2. 적용
tokenized_datasets = tokenized_datasets.cast(features)

Casting the dataset:   0%|          | 0/15005 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/3737 [00:00<?, ? examples/s]

In [20]:
tokenized_datasets['train'][0]

{'input_ids': [2, 2458, 15751, 24930, 24351, 29278, 17038, 11631, 3],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [0.0, 0.0, 1.0]}

In [21]:
# 데이터셋 포맷을 'touch'타입으로 변환 -> 학습에 사용할 수 있게 준비
tokenized_datasets.set_format(
    type = 'torch', # torch 타입으로 변경
)

In [22]:
tokenized_datasets['train'][0]

{'input_ids': tensor([    2,  2458, 15751, 24930, 24351, 29278, 17038, 11631,     3]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'labels': tensor([0., 0., 1.])}

### 사전학습 된 모델 로딩
- 다운스트림 태스크를 위한 사전학습 모델 불러오기!

In [23]:
# 2. 사전학습된 BERT기반 분류 모델(KcBERT 모델 로드)
model = BertForSequenceClassification.from_pretrained(
    model_name, # 모델 경로 지정
    num_labels = 3, # 클래스 개수
    problem_type = "multi_label_classification" # 각 라벨들을 독립적으로 0이나 1로 예측
    # 예를 들어, 문장이 혐오표현이면서 악플/욕설일 수도 있음 [1.0, 1.0, 0.0]
)

# single_label_classification : 단 하나의 라벨에 대해 예측

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# 3. DataCollactor 정의
batch_size = 64 # 배치 : 모델이 한번의 학습(훈련) 관정에서 처리할 데이터 수

# 데이터를 배치사이즈만큼 넣어주는 역할
# 자동으로 패딩도 수정
data_collator = DataCollatorWithPadding(tokenizer = tokenizer) # 데이터 토큰화 할 때 사용했던 토크나이저 지정

In [25]:
# 4. Training Arguments(학습 파라미터) 클래스 정의
args = TrainingArguments(
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = 5,
    output_dir = "./model_output",
    save_strategy =  "epoch", # 저장 전략
    load_best_model_at_end = True, # 학습이 끝날 때 가장 성능이 좋은 모델을 자동으로 로드
    greater_is_better = True, # 평가지표 기준으로 좋은 모델이라고 판단(순위 기반 평균 정밀도)
    eval_strategy =  "epoch", # 검증 전략, 검증을 진행할 타이밍 지정
    metric_for_best_model = "lrap" # 평가지표 지정
)

In [26]:
# LRAP 지표 불러오기
from sklearn.metrics import label_ranking_average_precision_score

# 순위 기반 평균 정밀도 : 다중 라벨 분류에서 사용되는 평가지표 중에 하나

# 모델이 A, B, C라는 3개의 독립적인 라벨에 대해 확률값을 예측한다고 가정
# A = 0.9, B = 0.6, C = 0.2
# 실제 라벨은 A와 C가 1이라고 가정, A는 확률이 높아서 순위가 1순위가 맞음, C는 확률이 낮아서 낮아서 잘 못 예측한 경우
# 이때의 정물도 값을 하나의 지표값으로 평가할 수 있는 도구

# 0 ~ 1, 모델이 각 샘플에 대해 라벨의 순위를 얼마나 잘 매겼는지 그 정도를 나타냄

In [30]:
# torch와의 호환을 위해 사용자 지정함수 정의

def my_metrics(x) :
  return {'lrap' : label_ranking_average_precision_score(x.label_ids, x.predictions)} # Changed x.label to x.label_ids

In [31]:
# 5. Trainer 클래스 -> 학습, 평가, 예측
trainer = Trainer(
    model = model, # 사전학습모델
    args = args, # 학습파라미터
    tokenizer = tokenizer, # 토크나이저
    data_collator = data_collator, # 데이터 콜렉터
    compute_metrics = my_metrics, # 평가지표 설정
    train_dataset = tokenized_datasets['train'], # 훈련데이터
    eval_dataset = tokenized_datasets['valid'], # 검증데이터
)

In [32]:
# 학습
trainer.train()

Epoch,Training Loss,Validation Loss,Lrap
1,No log,0.315166,0.898270
2,No log,0.359220,0.898939
3,0.162600,0.403849,0.899385
4,0.162600,0.474074,0.898225
5,0.058600,0.501916,0.897511


TrainOutput(global_step=1175, training_loss=0.09884223572751309, metrics={'train_runtime': 871.5663, 'train_samples_per_second': 86.081, 'train_steps_per_second': 1.348, 'total_flos': 2414765016563634.0, 'train_loss': 0.09884223572751309, 'epoch': 5.0})

### 새로운 문장에 대해 예측(추론)

In [36]:
!pip install torch
import torch

# 1. 입력 데이터
sentences = [
    '흑백요리사가 재밌더라 또 보고 싶었음', # 긍정
    '나의 한국 이름은 승"혁" 입니다.', # 중립
    '오늘 점심에는 김치전을 먹었는데 밀가루 맛이남.. 다음엔 안먹어야지', # 부정(악플/욕설)
    '핵 못 생겼음' # 부정(혐오표현)
]
# 2. 입력데이터에 대한 토크나이징
inputs = tokenizer(
    sentences,
    return_tensors = 'pt',
    truncation = True,
    padding = True,
    max_length = 512
)
# 3. 예측모드로 변경
# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

# 모델을 해당 디바이스로 이동
model.to(device)

# 입력 데이터도 해당 디바이스로 이동
inputs = {key : value.to(device) for key, value in inputs.items()}

# 모델을 평가 모드로 변경
model.eval()

# 4. 예측
with torch.no_grad() : # 기울기 계산 비활성화, 추론 속도가 향상
  outputs = model(**inputs) # 예측 결과를 얻음
  logits = outputs.logits # logit값으로 변환

In [37]:
# logits에 시그모이드 함수를 적용 -> 각 클래스에 대한 확률로 변환
probs = torch.sigmoid(logits)

# 임계값 기준으로 이진 분류(0.0, 1.0) 결과로 변환
threshold = 0.5
pred = (probs >= threshold).float() # 0.5 이상이라면 True(1.0), 그렇지 않으면 False(0.0)

In [38]:
label_names = ['혐오표현', '악플/욕설', 'clean']

for idx, sentence in enumerate(sentences):
    print(f"\n문장: {sentence}")
    print("예측 결과:")
    for i, label in enumerate(label_names):
        prediction = 'Yes' if pred[idx][i] == 1 else 'No'
        probability = probs[idx][i].item()
        print(f"  {label}: {prediction} (Probability: {probability:.4f})")


문장: 흑백요리사가 재밌더라 또 보고 싶었음
예측 결과:
  혐오표현: No (Probability: 0.0074)
  악플/욕설: No (Probability: 0.0122)
  clean: Yes (Probability: 0.9915)

문장: 나의 한국 이름은 승"혁" 입니다.
예측 결과:
  혐오표현: No (Probability: 0.0131)
  악플/욕설: No (Probability: 0.0159)
  clean: Yes (Probability: 0.9795)

문장: 오늘 점심에는 김치전을 먹었는데 밀가루 맛이남.. 다음엔 안먹어야지
예측 결과:
  혐오표현: No (Probability: 0.0060)
  악플/욕설: No (Probability: 0.0376)
  clean: Yes (Probability: 0.9738)

문장: 핵 못 생겼음
예측 결과:
  혐오표현: No (Probability: 0.0075)
  악플/욕설: No (Probability: 0.0128)
  clean: Yes (Probability: 0.9884)


In [39]:
torch.cuda.empty_cache()